In [149]:
!pip install transformers torch nltk flask


In [150]:
from transformers import pipeline

intent_model = pipeline("zero-shot-classification")

entity_model = pipeline("ner", aggregation_strategy="simple")


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializin

In [155]:
import json

def build_entity_types_mapping(dataset):
    entity_types = {}

    for entry in dataset:
        for entity_key, entity_value in entry['entities'].items():
            entity_type = entity_key
            if isinstance(entity_value, list):
                for value in entity_value:
                    if value not in entity_types:
                        entity_types[value] = entity_type
            else:
                if entity_value not in entity_types:
                    entity_types[entity_value] = entity_type

    return entity_types

with open('conver_data.json', 'r') as f:
    dataset = json.load(f)
entity_types = build_entity_types_mapping(dataset)

print(dataset)
print(entity_types)


[{'user_input': 'What are some must-read horror comics?', 'intent': 'recommend_comic', 'entities': {'genre': 'horror'}}, {'user_input': "I'm in the mood for adventure comics. Any suggestions?", 'intent': 'recommend_comic', 'entities': {'genre': 'adventure'}}, {'user_input': 'Can you recommend some heartfelt romance comics?', 'intent': 'recommend_comic', 'entities': {'genre': 'romance'}}, {'user_input': "I'd love to see some captivating melodrama comics. What do you suggest?", 'intent': 'recommend_comic', 'entities': {'genre': 'melodrama'}}, {'user_input': 'What are some intriguing fiction comics I should check out?', 'intent': 'recommend_comic', 'entities': {'genre': 'fiction'}}, {'user_input': 'Are there any great non-fiction comics you can recommend?', 'intent': 'recommend_comic', 'entities': {'genre': 'non-fiction'}}, {'user_input': 'I’m interested in fantasy comics. What do you recommend?', 'intent': 'recommend_comic', 'entities': {'genre': 'fantasy'}}, {'user_input': 'Suggest some

In [156]:
def extract_intent(user_input):
    labels = [entry['intent'] for entry in dataset]
    result = intent_model(user_input, candidate_labels=labels)
    return result

def extract_entities(user_input):
    entities = entity_model(user_input)
    annotated_entities = []
    for entity in entities:
        word = entity['word']
        entity_type = entity_types.get(word, 'unknown')
        start_position = entity['start']
        end_position = entity['end']
        annotated_entities.append({
            "word": word,
            "score": entity['score'],
            "type": entity_type,
            "start": start_position,
            "end": end_position
        })

    return annotated_entities

feedback_store = []

def collect_feedback(user_input, rating):
    feedback_store.append({"input": user_input, "rating": rating})
    save_feedback()

def save_feedback():
    with open('feedback.json', 'w') as f:
        json.dump(feedback_store, f, indent=4)

def adjust_responses():
    for feedback in feedback_store:
        if feedback["rating"] < 3:
            print(f"Improve response for: {feedback['input']}")


In [157]:
def chatbot_response(user_input):
    intent_result = extract_intent(user_input)
    entities_result = extract_entities(user_input)
    best_intent = intent_result['labels'][0]
    best_score = intent_result['scores'][0]

    if best_intent == "recommend_comic":
        response = "Sure! Here are some comic recommendations for you based on your interests."
    elif best_intent == "recommend_movie":
        response = "Absolutely! I can suggest some great movies. What genre are you interested in?"
    elif best_intent == "greetings":
        response = "Hello! How can I assist you today?"
    else:
        response = "I'm not sure how to help with that. Could you please provide more details?"
    response += f"\nIntent: {best_intent}\nEntities: {entities_result}"
    return response


user_input = "Can you check some comics about Horror"
response = chatbot_response(user_input)
print(response)

rating = int(input("Rate the response from 1 to 5: "))
collect_feedback(user_input, rating)
adjust_responses()


Sure! Here are some comic recommendations for you based on your interests.
Intent: recommend_comic
Entities: [{'word': 'Horror', 'score': 0.47618768, 'type': 'unknown', 'start': 32, 'end': 38}]
Rate the response from 1 to 5: 5


In [154]:
user_input = input("Please type your question or request: ")
response = chatbot_response(user_input)
print(response)

rating = int(input("Rate the response from 1 to 5: "))
collect_feedback(user_input, rating)
adjust_responses()

Please type your question or request: How about movies in Korean
Absolutely! I can suggest some great movies. What genre are you interested in?
Intent: recommend_movie
Entities: [{'word': 'Korean', 'score': 0.99773115, 'type': 'language', 'start': 20, 'end': 26}]
Rate the response from 1 to 5: 2
Improve response for: How about movies in Korean
